In [1]:
from data_pipeline import GHCN
import time
import warnings


In [2]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=30)


In [3]:
for i in range(365):
    print("Iter: ", i)
    t = time.time()
    msk_lap, x = ghcn[i]
    print("Time Taken: ", time.time() - t)


Iter:  0
---previous Laplacian expired or no previous Laplacian; building new---
Found 13204 stations in total in date range 20180101-20180130
Successfully converged after 110 steps
Difference between ML @ v and lambda * v:  6.729554983291283
Time Taken:  16.56012988090515
Iter:  1
Time Taken:  0.21923613548278809
Iter:  2
Time Taken:  0.22147297859191895
Iter:  3
Time Taken:  0.2723081111907959
Iter:  4
Time Taken:  0.21637487411499023
Iter:  5
Time Taken:  0.21394896507263184
Iter:  6
Time Taken:  0.21985316276550293
Iter:  7
Time Taken:  0.2807149887084961
Iter:  8
Time Taken:  0.2182769775390625
Iter:  9
Time Taken:  0.21566176414489746
Iter:  10
Time Taken:  0.244492769241333
Iter:  11
Time Taken:  0.2769148349761963
Iter:  12
Time Taken:  0.2192220687866211
Iter:  13
Time Taken:  0.22081995010375977
Iter:  14
Time Taken:  0.27321505546569824
Iter:  15
Time Taken:  0.2249751091003418
Iter:  16
Time Taken:  0.21890497207641602
Iter:  17
Time Taken:  0.21532702445983887
Iter:  18
Ti

Time Taken:  0.2317812442779541
Iter:  160
Time Taken:  0.3010721206665039
Iter:  161
Time Taken:  0.23349714279174805
Iter:  162
Time Taken:  0.22560524940490723
Iter:  163
Time Taken:  0.24244403839111328
Iter:  164
Time Taken:  0.2892570495605469
Iter:  165
Time Taken:  0.23970890045166016
Iter:  166
Time Taken:  0.23662590980529785
Iter:  167
Time Taken:  0.28991079330444336
Iter:  168
Time Taken:  0.2519040107727051
Iter:  169
Time Taken:  0.2224562168121338
Iter:  170
Time Taken:  0.21657586097717285
Iter:  171
Time Taken:  0.2799081802368164
Iter:  172
Time Taken:  0.22763609886169434
Iter:  173
Time Taken:  0.2251880168914795
Iter:  174
Time Taken:  0.24007916450500488
Iter:  175
Time Taken:  0.28493833541870117
Iter:  176
Time Taken:  0.24545621871948242
Iter:  177
Time Taken:  0.24399495124816895
Iter:  178
Time Taken:  0.38697004318237305
Iter:  179
Time Taken:  0.30512309074401855
Iter:  180
---previous Laplacian expired or no previous Laplacian; building new---
Found 13423

Time Taken:  0.2168867588043213
Iter:  322
Time Taken:  0.2644209861755371
Iter:  323
Time Taken:  0.2091379165649414
Iter:  324
Time Taken:  0.2086620330810547
Iter:  325
Time Taken:  0.22250699996948242
Iter:  326
Time Taken:  0.268064022064209
Iter:  327
Time Taken:  0.21751976013183594
Iter:  328
Time Taken:  0.2234361171722412
Iter:  329
Time Taken:  0.2187480926513672
Iter:  330
---previous Laplacian expired or no previous Laplacian; building new---
Found 13050 stations in total in date range 20181127-20181226
Successfully converged after 104 steps
Difference between ML @ v and lambda * v:  6.315486155529288
Time Taken:  17.731873035430908
Iter:  331
Time Taken:  0.21935486793518066
Iter:  332
Time Taken:  0.2144179344177246
Iter:  333
Time Taken:  0.26947879791259766
Iter:  334
Time Taken:  0.21227407455444336
Iter:  335
Time Taken:  0.22907686233520508
Iter:  336
Time Taken:  0.22519588470458984
Iter:  337
Time Taken:  0.2684330940246582
Iter:  338
Time Taken:  0.21295475959777

In [3]:
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix
import numpy as np


def mask_laplacian(laplacian, column_indices):
    """
    THIS creates a deep copy of L before doing inplace masking
    :param laplacian: COO-format sparse matrix
    :param column_indices: vertices to mask
    :return a COO-format sparse Laplacian
    """
    assert column_indices.dtype == np.long
    assert laplacian.format == "coo"
    laplacian_sparse = laplacian.tocsr()  # this actually creates a deep copy if L is not already csr
    assert laplacian_sparse is not laplacian
    I = sparse.identity(laplacian.shape[0], format="csr")
    I[column_indices, column_indices] = 0.0
    laplacian_sparse = laplacian_sparse @ I
    laplacian_sparse = laplacian_sparse.tolil()
    laplacian_sparse.setdiag(0.0)
    laplacian_sparse.setdiag(-np.asarray(laplacian_sparse.sum(axis=1)).squeeze())
    laplacian = laplacian_sparse.tocoo()
    return laplacian

import numpy as np
import scipy.stats as stats
import scipy.sparse as sparse
import matplotlib.pyplot as plt
np.random.seed((3,14159))

def sprandsym(n, density):
    rvs = stats.norm().rvs
    X = sparse.random(n, n, density=density, data_rvs=rvs)
    upper_X = sparse.triu(X) 
    result = upper_X + upper_X.T - sparse.diags(X.diagonal())
    return result


In [24]:
A = sprandsym(4, 0.6)
print(A.todense())
A.setdiag(np.zeros(A.shape[0]))
diag = np.asarray(np.sum(A, axis=1)).squeeze()
A.setdiag(-diag)
print(A.todense())
np.sum(A, axis=1)


[[ 0.48346935  0.         -0.74821355  0.        ]
 [ 0.          0.          0.         -0.09495038]
 [-0.74821355  0.         -1.9424273   0.        ]
 [ 0.         -0.09495038  0.          0.81440027]]
[[ 0.74821355  0.         -0.74821355  0.        ]
 [ 0.          0.09495038  0.         -0.09495038]
 [-0.74821355  0.          0.74821355  0.        ]
 [ 0.         -0.09495038  0.          0.09495038]]


/Users/dwd/anaconda3/lib/python3.6/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


matrix([[0.],
        [0.],
        [0.],
        [0.]])

In [25]:
masked_A = mask_laplacian(A.tocoo(), np.array([1, 2]))
masked_A


<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in COOrdinate format>

In [26]:
masked_A.todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.09495038,  0.        , -0.09495038],
        [-0.74821355,  0.        ,  0.74821355,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [23]:
import numpy.linalg as npla
eigvals = npla.eigvals(masked_A.todense())
eigvals


array([ 2.01601381e-01, -4.99961057e-01,  2.79473773e+00,  2.22044605e-16])